In [1]:
! pip install --quiet langchain pymongo langchain-openai langchain-community pypdf

In [2]:
import os
import config
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import MongoDBAtlasVectorSearch
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from pymongo import MongoClient
from langchain_community.document_loaders import DirectoryLoader

# Set up your OpenAI API key
os.environ["OPENAI_API_KEY"] = config.OPENAI_API_KEY

# Set up MongoDB connection
mongo_uri = config.MONGO_URI
db_name = "carlisle"
collection_name = "research"

client = MongoClient(mongo_uri, appname="devrel.showcase.chat_with_pdf")
db = client[db_name]
collection = db[collection_name]

# Set up document loading and splitting
loader = DirectoryLoader('./test_files', glob="**/*.md")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

# Set up embeddings and vector store
embeddings = OpenAIEmbeddings()
vector_store = MongoDBAtlasVectorSearch.from_documents(
    texts,
    embeddings,
    collection=collection,
    index_name="vector_index"
)

# Set up retriever and language model
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Set up RAG pipeline
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

# Function to process user query
def process_query(query):
    result = qa_chain({"query": query})
    return result["result"], result["source_documents"]

# Example usage
query = "How do I reset my device"
answer, sources = process_query(query)
print(f"Answer: {answer}")
print("Sources:")
for doc in sources:
    print(f"- {doc.metadata['source']}: {doc.page_content[:100]}...")

# Don't forget to close the MongoDB connection when done
client.close()

/var/folders/lg/3y44c83j6x12y235mw36jdfm0000gn/T/ipykernel_76673/76384287.py:32: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()
/var/folders/lg/3y44c83j6x12y235mw36jdfm0000gn/T/ipykernel_76673/76384287.py:42: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
/var/folders/lg/3y44c83j6x12y235mw36j

Answer: I don't know which device you are referring to. Could you please specify the type of device you are using so I can provide you with the correct instructions on how to reset it?
Sources:
